<a href="https://colab.research.google.com/github/sunilkumar1762/BIDEN_AI/blob/main/AI_BIDEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALLING AND IMPORTING NECCESSARY LIBRARIES


In [ ]:
pip install transformers datasets pandas torch


In [ ]:
!pip uninstall numpy
!pip install numpy

Found existing installation: numpy 1.25.2
Uninstalling numpy-1.25.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.10
    /usr/local/lib/python3.10/dist-packages/numpy-1.25.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libgfortran-040039e1.so.5.0.0
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libopenblas64_p-r0-5007b62f.3.23.dev.so
    /usr/local/lib/python3.10/dist-packages/numpy.libs/libquadmath-96973f99.so.0.0.0
    /usr/local/lib/python3.10/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-1.25.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 41.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
astropy 5.3.4 requires numpy<2,>=1.21, but you have numpy 2.0.0 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but yo

In [ ]:
pip install tensorflow


  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.22.0 requires numpy==1.22.0; python_version <= "3.10", but you have numpy 1.26.4 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.4 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U


DEVELOPING A PRE_TRAINED MODEL


In [ ]:
import subprocess
import sys

# Ensure accelerate is installed
subprocess.check_call([sys.executable, "-m", "pip", "install", "accelerate>=0.21.0"])

import pandas as pd
from datasets import Dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch

# Load the dataset
file_path = '/content/biden dataset.xlsx'
data = pd.read_excel(file_path)

# Ensure the dataset contains the required columns
if 'Speaker' not in data.columns or 'Speech' not in data.columns:
    raise ValueError("Dataset must contain 'Speaker' and 'Speech' columns")

# Convert 'Speaker' column to numerical labels
data['label'] = data['Speaker'].astype('category').cat.codes
label_map = dict(enumerate(data['Speaker'].astype('category').cat.categories))
num_labels = len(label_map)

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['Speech'], padding="max_length", truncation=True)

# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Ensure the dataset includes labels
tokenized_dataset = tokenized_dataset.map(lambda examples: {'labels': examples['label']}, batched=True)

# Define the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('path_to_save_model')
tokenizer.save_pretrained('path_to_save_model')

# Example prediction with the fine-tuned model
def predict(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    predictions = outputs.logits.argmax(-1)
    return label_map[predictions.item()]


example_text = "Thank you for having me"
prediction = predict(example_text)
print(f"Prediction for '{example_text}': {prediction}")

example_text = "Mister President, thank you for doing this."
prediction = predict(example_text)
print(f"Prediction for '{example_text}': {prediction}")

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.673078
2,No log,0.659731
3,No log,0.653045


Prediction for 'Thank you for having me': Biden
Prediction for 'Mister President, thank you for doing this.': Interviewer


Set Up Text-to-Speech (TTS)

In [ ]:
!pip install TTS
!pip install soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 55.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) 

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [ ]:
import pandas as pd
from TTS.api import TTS

# Load the Excel file
file_path = '/content/biden dataset.xlsx'
data = pd.read_excel(file_path)

# Initialize the TTS model
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC")

# Function to preprocess text
def preprocess_text(text):
    # Example preprocessing steps
    text = text.replace("\n", " ")  # Remove newline characters
    text = text.strip()  # Remove leading and trailing whitespace
    return text

# Iterate through each row in the dataset and synthesize the speech
for index, row in data.iterrows():
    text = preprocess_text(row['Speech'])
    file_name = f"speech_{index}.wav"
    file_path = f"output"  # Update this path

    # Generate speech and save to file
    tts.tts_to_file(text=text, file_path=file_path)

print("Speech synthesis complete.")


 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P

VIDEO GENERATION


In [ ]:

!pip install moviepy

from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

# Load the video (replace 'your_video.mp4' with your actual video file)
video_clip = VideoFileClip("/content/bidenn.mp4")

# Load the generated audio
audio_clip = AudioFileClip("/content/biden voice.mp3")

# Ensure the audio clip is not longer than the video
audio_clip = audio_clip.subclip(0, video_clip.duration)

# Combine the audio and video
final_clip = video_clip.set_audio(audio_clip)

# Save the final video (replace 'output_video.mp4' with your desired output filename)
final_clip.write_videofile("output_video.mp4")




100%|██████████| 3.80M/3.80M [14:58<00:00, 34.1MiB/s]

100%|██████████| 3.80M/3.80M [14:58<00:00, 34.1MiB/s]

Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


                                                                 

100%|██████████| 3.80M/3.80M [14:59<00:00, 34.1MiB/s]

100%|██████████| 3.80M/3.80M [14:59<00:00, 34.1MiB/s]

MoviePy - Done.
Moviepy - Writing video output_video.mp4





100%|██████████| 3.80M/3.80M [15:05<00:00, 34.1MiB/s]

100%|██████████| 3.80M/3.80M [15:05<00:00, 34.1MiB/s]

Moviepy - Done !
Moviepy - video ready output_video.mp4


In [ ]:
import pandas as pd

# Load the Excel file
file_path = '/content/biden dataset.xlsx'
data = pd.read_excel(file_path)

# Preprocess the dataset to create a dictionary of questions and video file paths
responses = {}
for index, row in data.iterrows():
    question = row['Speech']
    video_file = f"/content/bidenn.mp4"  # Assume the videos are named accordingly
    responses[question] = video_file

# Print the preprocessed responses to verify
responses


{'Mister President, thank you for doing this.': '/content/bidenn.mp4',
 'Thank you for having me.': '/content/bidenn.mp4',
 "Let's start with the debate": '/content/bidenn.mp4',
 'You and your team have said you had a bad night, but your friend Nancy Pelosi actually framed the question that I think is on the minds of millions of Americans.': '/content/bidenn.mp4',
 'Was this a bad episode or the sign of a more serious condition?': '/content/bidenn.mp4',
 'It was a bad episode,': '/content/bidenn.mp4',
 'No indicated any serious condition.': '/content/bidenn.mp4',
 'I was exhausted.': '/content/bidenn.mp4',
 "I didn't listen to my instincts in terms of preparing and bad nights.": '/content/bidenn.mp4',
 " You know, you say you were exhausted and I know you've said that before as well, but you came and       you did have a tough month, but you came home from Europe about eleven or twelve days before the debate, spent six days in Camp David,": '/content/bidenn.mp4',
 "Why wasn't that enou

CREATING A TELEGRAM BOT

In [ ]:
!pip install telebot
import telebot

# Replace with your bot token
bot_token = '7384414112:AAEmePSiE-EK-jr73BSxuQsdBNCHbzWR8LA'
bot = telebot.TeleBot(bot_token)

# Load the Excel file to create the responses dictionary
import pandas as pd
file_path = '/content/biden dataset.xlsx'
data = pd.read_excel(file_path)
responses = {}
for index, row in data.iterrows():
    question = row['Speech']
    video_file = f"/content/bidenn.mp4"
    responses[question] = video_file

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Hi! Ask me anything.")

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    user_prompt = message.text
    response_video = find_response(user_prompt)

    if response_video:
        video_path = f"/content/bidenn.mp4"  # Update this path
        bot.send_video(message.chat.id, video=open(video_path, 'rb'))
    else:
        bot.reply_to(message, "I don't have a response for that.")

def find_response(user_prompt):
    # Access the responses dictionary correctly
    for question, video_file in responses.items():  # Removed incorrect argument
        if user_prompt.lower() in question.lower():  # Removed incorrect argument
            return video_file
    return None

bot.polling()